In [73]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


import random
random.seed(42)

In [74]:
data = pd.read_csv('data.csv')
texts = data['text'].tolist()
labels = data['label'].tolist()

In [75]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [76]:
max_len = 256
X = pad_sequences(sequences, maxlen=max_len)

In [77]:
y = np.array(labels)

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [79]:
model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=max_len),
    LSTM(256, return_sequences=False),
    Dropout(0.5),
    Dense(128 , activation='relu'),
    Dense(1, activation='sigmoid')
])

In [80]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [81]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [82]:
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=16,
    validation_split=0.2,
    callbacks=[early_stopping]
)

Epoch 1/10
223/223 [==============================] - 7s 25ms/step - loss: 0.1193 - accuracy: 0.9647 - val_loss: 0.0487 - val_accuracy: 0.9865
Epoch 2/10
223/223 [==============================] - 5s 24ms/step - loss: 0.0311 - accuracy: 0.9927 - val_loss: 0.0380 - val_accuracy: 0.9854
Epoch 3/10
223/223 [==============================] - 5s 24ms/step - loss: 0.0113 - accuracy: 0.9972 - val_loss: 0.0301 - val_accuracy: 0.9899
Epoch 4/10
223/223 [==============================] - 5s 24ms/step - loss: 0.0033 - accuracy: 0.9989 - val_loss: 0.0346 - val_accuracy: 0.9888
Epoch 5/10
223/223 [==============================] - 5s 23ms/step - loss: 8.6962e-04 - accuracy: 1.0000 - val_loss: 0.0333 - val_accuracy: 0.9933
Epoch 6/10
223/223 [==============================] - 5s 24ms/step - loss: 2.2125e-04 - accuracy: 1.0000 - val_loss: 0.0337 - val_accuracy: 0.9933


In [83]:
loss_train , accuracy_train = model.evaluate(X_train, y_train)
print(f"Training Accuracy: {accuracy_train:.4f}")

140/140 [==============================] - 1s 10ms/step - loss: 0.0091 - accuracy: 0.9971
Training Accuracy: 0.9971


In [84]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

35/35 [==============================] - 0s 10ms/step - loss: 0.0512 - accuracy: 0.9883
Test Accuracy: 0.9883


In [85]:
def predict_spam(texts):
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, maxlen=max_len)

    predictions = model.predict(padded_sequences)

    return ['Spam' if pred > 0.5 else 'Not Spam' for pred in predictions]

In [86]:
text = ["Congratulations! You've won a free ticket to the cinema. Call now to claim your prize!"]
predictions = predict_spam(text)
for t, label in zip(text, predictions):
    print(f"Text: {t}\nPredicted Label: {label}\n")

1/1 [==============================] - 0s 250ms/step
Text: Congratulations! You've won a free ticket to the cinema. Call now to claim your prize!
Predicted Label: Spam



In [87]:
model.save('model.h5')

In [88]:
from joblib import dump
with open('tokenizer.h5' , 'wb') as f:
    dump(tokenizer, f)

In [89]:
import json
params = {
    'max_len' : max_len,
    'num_words' : 5000,
}

with open('params.json', 'w') as f:
    json.dump(params, f)